In [1]:
import os
for dirname, _, filenames in os.walk('/brain/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np




In [4]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Test data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)



In [17]:
# Load and preprocess your dataset (adjust paths and parameters)
train_generator = train_datagen.flow_from_directory(
    "D:/TAHAB WORKING FOLDER/Prototype_MajorProject/1/New folder/brain/Training",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
   "D:/TAHAB WORKING FOLDER/Prototype_MajorProject/1/New folder/brain/Testing",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)



Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [21]:
test_generator.class_indices

{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}

In [7]:
# Pre-trained InceptionV3 model as the base

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



In [8]:
# Fine-tuning the last layers of the base model
for layer in base_model.layers[:-10]:
    layer.trainable = False


In [10]:
# Create your model
model = Sequential()
model.add(base_model)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))


In [11]:

# Compile the model
model.compile(optimizer=Adam(lr=0.00005),  # Experiment with different learning rates
              loss='categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\MUZAMMIL\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



In [7]:
import tensorflow 


In [13]:
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
# Train the model with early stopping/////, callbacks=[tensorboard_callback]
model.fit(train_generator, epochs=100, validation_data=test_generator)


In [10]:
model = tensorflow.keras.models.load_model("braintumor.h5",compile = False) 

In [11]:
model.compile(optimizer=Adam(lr=0.00005),  # Experiment with different learning rates
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\MUZAMMIL\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

41/41 [==============================] - 231s 5s/step - loss: 0.4222 - accuracy: 0.8772
Test Loss: 0.4222
Test Accuracy: 0.8772


In [ ]:
model.save('model.h5')